<div style="text-align: center;">
    <img src="https://rubikcloud.ai/wp-content/uploads/2024/03/logo-bottom.png" alt="Image" width="200"/>
</div>


<p align="center">
  <strong style="font-size: 44px;">Trabajo Practico 2: Reconstruccion 3D</strong><br>
  <em style="font-size: 16px;">Codigo de Bruno Castagnino Rossi, Olivia Browne Corbacho y Agustin Federico Manzano - Parte 1</em>
</p>

In [1]:
import pickle as pkl
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import os 
import sys

In [2]:
from code import initiation

path1 = 'data/captures/left_3.jpg'
path2 = 'data/captures/left_3.jpg'

pklpath = 'data/stereo_maps.pkl'

imgL, imgR, mat_dict = initiation(path1, path2, pklpath)

In [3]:
print(mat_dict['Q'])

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -9.70262848e+02]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -5.39484238e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.00600906e+02]
 [ 0.00000000e+00  0.00000000e+00  1.66848275e-02 -0.00000000e+00]]


In [4]:
from code import show_rectified_images
show_rectified_images(imgL, imgR, mat_dict, plot_images=True)

(array([[[ 41,  40,  26],
         [ 41,  39,  25],
         [ 40,  38,  26],
         ...,
         [214, 201, 185],
         [215, 201, 185],
         [215, 201, 185]],
 
        [[ 43,  40,  26],
         [ 42,  39,  25],
         [ 41,  38,  25],
         ...,
         [216, 201, 185],
         [216, 201, 185],
         [216, 201, 185]],
 
        [[ 43,  40,  26],
         [ 42,  39,  26],
         [ 42,  38,  26],
         ...,
         [216, 201, 185],
         [216, 201, 185],
         [216, 201, 185]],
 
        ...,
 
        [[145, 159, 158],
         [145, 158, 158],
         [145, 157, 157],
         ...,
         [ 90,  59,  44],
         [ 90,  59,  44],
         [ 89,  58,  43]],
 
        [[145, 159, 157],
         [145, 158, 157],
         [145, 158, 156],
         ...,
         [ 90,  60,  45],
         [ 90,  59,  44],
         [ 90,  59,  44]],
 
        [[146, 160, 156],
         [146, 159, 156],
         [145, 160, 156],
         ...,
         [ 87,  58,  42],
  

In [5]:
from code import save_imgs

save_imgs(mat_dict)
    

Rectified images saved to data/rectified_images
Rectified images 0 saved
Rectified images saved to data/rectified_images
Rectified images 1 saved
Rectified images saved to data/rectified_images
Rectified images 2 saved
Rectified images saved to data/rectified_images
Rectified images 3 saved
Rectified images saved to data/rectified_images
Rectified images 4 saved
Rectified images saved to data/rectified_images
Rectified images 5 saved
All images saved


In [6]:
#utilizamos stereodemo para ver las imagenes rectificadas
